In [ ]:
import sys
sys.path.append('/Users/kevin/Projects/CS236_Course_Project')

In [ ]:
import plotly.graph_objs as go
import numpy as np
import torch

from src.model import Generator
from src.trainer import Trainer
from src.metrics import compute_metrics

In [ ]:
BATCH_SIZE = 1
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
CKPT_PATH = "/Users/kevin/Projects/CS236_Course_Project/checkpoints/sinkhorn_energy_gaussian_laplacian_2023-12-02_13-49-52/1900.pth"

In [ ]:
# Load point cloud file
example_pc_path = "/Users/kevin/Projects/CS236_Course_Project/mock_data/Test/451927.8000000001_453201.865.npy"
real_point_cloud = torch.from_numpy(np.load(example_pc_path)).unsqueeze(0)

In [ ]:
def visualize_point_cloud(data: np.ndarray):
    # Assuming your data is a NumPy array of shape [1000, 3]
    # Create 3D scatter plot
    trace = go.Scatter3d(
        x=data[:, 0],
        y=data[:, 1],
        z=data[:, 2],
        mode='markers',
        marker=dict(
            size=3,
            color=data[:, 2],  # You can use another column for color
            colorscale='Viridis',
            opacity=0.8
        )
    )

    # Create layout
    layout = go.Layout(scene=dict(aspectmode='data'))

    # Create figure
    fig = go.Figure(data=[trace], layout=layout)

    # Show the plot
    fig.show()

In [ ]:
# Load trained generator model checkpoint 

# Setup model
net_g = Generator()
net_g.eval()

# Setup trainer
trainer = Trainer(net_g=net_g, batch_size=BATCH_SIZE, device=DEVICE)

# Load checkpoint
trainer.load_checkpoint(CKPT_PATH)

### TBD what is the correct approach:

net_g.decode() adds random noise to the encoded latent point cloud to generate new point cloud. We do this in an attempt to learn a meaningful latent space representation of point clouds. However, during inference time, my hypothesis would be that we do not add random noise and just generate point clouds from the interpolated latents directly (as indicated in the cell below)

In [ ]:
# Example taken from Trainer.test()
generated_point_cloud, latent_point_cloud = net_g(real_point_cloud)
# generated_point_cloud has shape: torch.Size([1, 500, 3])
metrics_original = compute_metrics(generated_point_cloud, real_point_cloud, BATCH_SIZE)
generated_point_cloud = generated_point_cloud.squeeze().detach().numpy()

print(f"Original Metrics: {metrics_original}")
visualize_point_cloud(generated_point_cloud)

In [ ]:
# My idea
# Wait, net_g.decode() also adds noise to the latent point cloud. So, we should just use the latent point cloud to generate the point cloud.
generated_output = net_g.decode(latent_point_cloud, 1, 500, DEVICE)
metrics_output = compute_metrics(generated_output, real_point_cloud, BATCH_SIZE)
generated_output = generated_output.squeeze().detach().numpy()
print(f"Output Metrics: {metrics_output}")
visualize_point_cloud(generated_output)